<a href="https://colab.research.google.com/github/sharmaraja/AI-Tools/blob/main/Stock_data_API_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API Testing:
- Hit APIs and check data
- See what all we can get from it

In [ ]:
!pip install tradingview-screener, apify-client

In [12]:
# pip install python-dotenv requests
import os
import requests
from dotenv import load_dotenv
from datetime import datetime

import pandas as pd
from apify_client import ApifyClient

**Code for Apify Data**

In [10]:
apify_key = userdata.get('APIFY_API_KEY')
if apify_key:
    print("APIFY_API_KEY value fetched")

APIFY_API_KEY value fetched


In [13]:
# 1. Initialize the Apify Client
client = ApifyClient(apify_key)

In [16]:
# 2. Prepare the Actor input
# Replace 'TATASTEEL' with any Indian stock ticker or Screener URL
# run_input = {
#     "startUrls": [{"url": "https://www.screener.in"}],
#     "maxPagesPerCrawl": 1
# }

run_input = {
    "mode": "getstockdetails",
    "url": "https://www.screener.in/company/TATASTEEL/consolidated/",
    "maxPagesPerCrawl": 1
}

# 3. Run the Actor and wait for it to finish
print("Starting data extraction...")
run = client.actor("shashwattrivedi/screener-in").call(run_input=run_input)

# 4. Fetch the results from the dataset
dataset_items = list(client.dataset(run["defaultDatasetId"]).iterate_items())

if not dataset_items:
    print("No data found.")
else:
    # 5. Extract specific statements into a DataFrame
    # The actor typically returns a single item containing nested financial tables
    stock_data = dataset_items[0]

    # helper function to format nested dictionary tables into DataFrames
    def get_statement_df(data, key_name):
        if key_name in data and data[key_name]:
            # Screener data is usually a list of rows
            df = pd.DataFrame(data[key_name])

            # The first column usually contains the metric names (Sales, Expenses, etc.)
            if not df.empty:
                # Dynamically find the name of the first column (e.g., 'Unnamed: 0' or 'Report Date')
                first_col = df.columns[0]
                df.set_index(first_col, inplace=True)
            return df
        return pd.DataFrame()

    # The Actor keys might use underscores based on the logs
    df_pl = get_statement_df(stock_data, "profitAndLoss")
    df_bs = get_statement_df(stock_data, "balanceSheet")
    df_cf = get_statement_df(stock_data, "cashFlow")

    # Display results
    print("\n--- Profit & Loss Statement ---")
    print(df_pl.head())

    print("\n--- Balance Sheet ---")
    print(df_bs.head())

    print("\n--- Cash Flow Statement ---")
    print(df_cf.head())

Starting data extraction...


[apify.screener-in runId:qijMNhfQYFuPg8e46] -> Status: RUNNING, Message: 
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:29.824Z ACTOR: Pulling container image of build 1XmCLWfkIoPCAnUl8 from registry.
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:48.666Z ACTOR: Creating container.
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:48.734Z ACTOR: Starting container.
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:48.958Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" python3 -m src
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:50.456Z Actor is running on the Apify platform, `disable_browser_sandbox` was changed to True.
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:50.636Z [apify] INFO  Initializing Actor...
[apify.screener-in runId:qijMNhfQYFuPg8e46] -> 2026-02-16T03:56:50.640Z [apify] INFO  System info ({"apify_sdk_version": "2.7.3", "apify_client_versio


--- Profit & Loss Statement ---
Empty DataFrame
Columns: []
Index: []

--- Balance Sheet ---
Empty DataFrame
Columns: []
Index: []

--- Cash Flow Statement ---
Empty DataFrame
Columns: []
Index: []


In [18]:
dataset_items[0]

{'company_name': 'Tata Steel Ltd',
 'peers': [{},
  {'median_summary': [None,
    'Median: 13 Co.',
    132.75,
    21.23,
    11722.42,
    0.23,
    86.85,
    30.02,
    1353.35,
    9.82,
    8.83]}],
 'quarters': [{'Metric': 'Sales',
   'Dec 2022': 57084,
   'Mar 2023': 62962,
   'Jun 2023': 59490,
   'Sep 2023': 55682,
   'Dec 2023': 55312,
   'Mar 2024': 58687,
   'Jun 2024': 54771,
   'Sep 2024': 53905,
   'Dec 2024': 53648,
   'Mar 2025': 56218,
   'Jun 2025': 53178,
   'Sep 2025': 58689,
   'Dec 2025': 57002},
  {'Metric': 'Expenses',
   'Dec 2022': 53036,
   'Mar 2023': 55742,
   'Jun 2023': 54587,
   'Sep 2023': 51414,
   'Dec 2023': 49048,
   'Mar 2024': 52087,
   'Jun 2024': 48077,
   'Sep 2024': 47789,
   'Dec 2024': 47745,
   'Mar 2025': 49659,
   'Jun 2025': 45751,
   'Sep 2025': 49793,
   'Dec 2025': 48803},
  {'Metric': 'Operating Profit',
   'Dec 2022': 4048,
   'Mar 2023': 7219,
   'Jun 2023': 4903,
   'Sep 2023': 4268,
   'Dec 2023': 6264,
   'Mar 2024': 6601,
   

In [19]:
# 1. Extract the main dictionary
stock_info = dataset_items[0]

def json_to_dataframe(source_data):
    """Converts the 'Metric' style list of dicts into a clean DataFrame"""
    if not source_data:
        return pd.DataFrame()

    df = pd.DataFrame(source_data)
    # Set the 'Metric' column as the row names (index)
    if 'Metric' in df.columns:
        df.set_index('Metric', inplace=True)
    return df

# 2. Extract specific statements
# Profit & Loss (extracted from the 'annual_data' sub-key)
df_pl = json_to_dataframe(stock_info.get('profit_and_loss', {}).get('annual_data', []))

# Balance Sheet
df_bs = json_to_dataframe(stock_info.get('balance_sheet', []))

# Cash Flow (The key from your previous log was 'cash_flow')
df_cf = json_to_dataframe(stock_info.get('cash_flow', []))

# 3. Clean up: Remove any empty columns/rows if they exist
# df_pl = df_pl.dropna(how='all', axis=1)

In [20]:
df_pl

,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Mar 2025,TTM
Metric,,,,,,,,,,,,,
Sales,148614.00,139504.00,101965.00,112299.00,123249.00,157669.00,139817.00,156294.00,243959.00,243353.00,229171.00,218543.00,225088.00
Expenses,132430.00,136095.00,102590.00,99156.00,101816.00,128351.00,122354.00,125790.00,180469.00,211053.00,206923.00,193244.00,194005.00
Operating Profit,16184.00,3409.00,-625.00,13144.00,21433.00,29318.00,17463.00,30504.00,63490.00,32300.00,22248.00,25298.00,31083.00
OPM %,11.00,2.00,-1.00,12.00,17.00,19.00,12.00,20.00,26.00,13.00,10.00,12.00,14.00
Other Income,716.00,5994.00,10346.00,-3789.00,10898.00,1524.00,-2884.00,180.00,1300.00,1569.00,-6005.00,877.00,845.00
Interest,4337.00,4848.00,4221.00,5072.00,5455.00,7660.00,7533.00,7607.00,5462.00,6299.00,7508.00,7341.00,7163.00
Depreciation,5841.00,5944.00,5306.00,5673.00,5742.00,7342.00,8441.00,9234.00,9101.00,9335.00,9882.00,10421.00,11406.00
Profit before tax,6722.00,-1388.00,193.00,-1391.00,21135.00,15841.00,-1395.00,13844.00,50227.00,18235.00,-1147.00,8413.00,13358.00
Tax %,45.00,185.00,358.00,200.00,16.00,42.00,-184.00,41.00,17.00,56.00,328.00,62.00,NaN


In [21]:
df_bs

,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Mar 2025,Sep 2025
Metric,,,,,,,,,,,,,
Equity Capital,971,971,970,970,1145,1145,1145,1198,1221,1221,1247,1247,1247
Reserves,39561,30378,42762,36849,59726,67780,72431,73041,113222,101861,90788,89922,93700
Borrowings,81629,80701,81987,83014,92147,100816,116328,88501,75561,84893,87082,94801,95643
Other Liabilities,49443,47059,51165,51614,55704,63032,59245,81169,92417,97421,90195,89488,94975
Total Liabilities,171604,159109,176884,172447,208722,232773,249149,243909,282422,285396,269312,275459,285565
Fixed Assets,74907,68100,72200,92007,96105,124442,134551,135775,133288,146621,148814,150914,166095
CWIP,26822,28678,35996,15784,16614,18641,19497,19007,22046,31213,34356,41622,33152
Investments,5093,3455,10714,12457,17899,5738,6285,10682,13140,8410,6258,6194,6903
Other Assets,64781,58876,57974,52200,78103,83952,88816,78445,113948,99151,79885,76728,79415


In [22]:
df_cf

,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Mar 2025
Metric,,,,,,,,,,,,
Cash from Operating Activity,13146,11880,11455,10824,8023,25336,20169,44327,44381,21683,20301,23512
Cash from Investing Activity,-15378,-9021,-8794,-9490,-11730,-29176,-14012,-9437,-10905,-18179,-14253,-13985
Cash from Financing Activity,1015,-2617,-4729,-2579,6640,-673,-1695,-37090,-23401,-6981,-11097,-7002
Net Cash Flow,-1218,241,-2068,-1245,2933,-4513,4462,-2200,10075,-3477,-5049,2524


In [25]:
# Assuming 'stock_data' is the dictionary you received
company_name = stock_data.get('company_name', 'N/A')
print(f"Company: {company_name}")

Company: Tata Steel Ltd


In [26]:
# Convert the 'quarters' list into a DataFrame
df_quarters = pd.DataFrame(stock_data.get('quarters', []))

# Set 'Metric' as the index so quarters (Dec 2022, etc.) are your columns
if not df_quarters.empty:
    df_quarters.set_index('Metric', inplace=True)

print("\n--- Quarterly Results ---")
print(df_quarters.head())



--- Quarterly Results ---
                  Dec 2022  Mar 2023  Jun 2023  Sep 2023  Dec 2023  Mar 2024  \
Metric                                                                         
Sales              57084.0   62962.0   59490.0   55682.0   55312.0   58687.0   
Expenses           53036.0   55742.0   54587.0   51414.0   49048.0   52087.0   
Operating Profit    4048.0    7219.0    4903.0    4268.0    6264.0    6601.0   
OPM %                  7.0      11.0       8.0       8.0      11.0      11.0   
Other Income         491.0     277.0    1190.0   -6568.0     -33.0    -382.0   

                  Jun 2024  Sep 2024  Dec 2024  Mar 2025  Jun 2025  Sep 2025  \
Metric                                                                         
Sales              54771.0   53905.0   53648.0   56218.0   53178.0   58689.0   
Expenses           48077.0   47789.0   47745.0   49659.0   45751.0   49793.0   
Operating Profit    6694.0    6116.0    5903.0    6559.0    7428.0    8897.0   
OPM %       

In [27]:
# Extract peer median data if available
peers_data = stock_data.get('peers', [])
if len(peers_data) > 1:
    median_summary = peers_data[1].get('median_summary', [])
    print(f"\nIndustry Median Stats: {median_summary}")


Industry Median Stats: [None, 'Median: 13 Co.', 132.75, 21.23, 11722.42, 0.23, 86.85, 30.02, 1353.35, 9.82, 8.83]


In [38]:
len(dataset_items)

1

In [45]:
# 1. FIX: QUARTERLY RESULTS DATAFRAME
df_quarters = pd.DataFrame(stock_data.get('quarters', []))
if not df_quarters.empty:
    df_quarters.set_index('Metric', inplace=True)
df_quarters.head()

,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Sep 2024,Dec 2024,Mar 2025,Jun 2025,Sep 2025,Dec 2025
Metric,,,,,,,,,,,,,
Sales,57084.0,62962.0,59490.0,55682.0,55312.0,58687.0,54771.0,53905.0,53648.0,56218.0,53178.0,58689.0,57002.0
Expenses,53036.0,55742.0,54587.0,51414.0,49048.0,52087.0,48077.0,47789.0,47745.0,49659.0,45751.0,49793.0,48803.0
Operating Profit,4048.0,7219.0,4903.0,4268.0,6264.0,6601.0,6694.0,6116.0,5903.0,6559.0,7428.0,8897.0,8200.0
OPM %,7.0,11.0,8.0,8.0,11.0,11.0,12.0,11.0,11.0,12.0,14.0,15.0,14.0
Other Income,491.0,277.0,1190.0,-6568.0,-33.0,-382.0,-6.0,617.0,142.0,149.0,236.0,-6.0,465.0


In [46]:
def get_latest_value(df, metric_name):
    """Safely extracts the most recent value from a metric row"""
    if metric_name in df.index:
        # Get the last non-null column value (the most recent period)
        row = df.loc[metric_name].dropna()
        return row.iloc[-1] if not row.empty else None
    return None

# 2. FIX: FUNDAMENTAL RATIOS (Extracting from Quarters/Annual if missing at top)
# Scrapers often don't provide a 'Stock PE' key; you calculate or find it in specific tables
fundamentals = {
    "Stock P/E": stock_data.get('stock_pe') or get_latest_value(df_quarters, 'P/E'),
    "Debt/Equity": stock_data.get('debt_to_equity'),
    "ROCE %": stock_data.get('roce'),
    "ROE %": stock_data.get('roe'),
    # Get 3Y growth from the 'profit_and_loss' -> 'Compounded Sales Growth' list
}

# 3. FIX: SHAREHOLDING PATTERN (FII/DII)
# The scraper provides a 'shareholding' key with 'Promoters', 'FIIs', 'DIIs' rows
sh_raw = stock_data.get('shareholding', [])
df_shareholding = pd.DataFrame(sh_raw)
if not df_shareholding.empty:
    # Use the first column (Metric/Name) as index dynamically
    df_shareholding.set_index(df_shareholding.columns[0], inplace=True)

# 4. FIX: PROS AND CONS
# These are usually direct lists in the JSON
pros_list = stock_data.get('pros', [])
cons_list = stock_data.get('cons', [])
df_pros_cons = pd.DataFrame({
    "Pros": pd.Series(pros_list),
    "Cons": pd.Series(cons_list)
})

# --- OUTPUT ---
print("--- FUNDAMENTAL RATIOS ---")
print(pd.DataFrame([fundamentals]))

print("\n--- SHAREHOLDING PATTERN ---")
print(df_shareholding if not df_shareholding.empty else "Empty")

print("\n--- PROS & CONS ---")
print(df_pros_cons.head())

--- FUNDAMENTAL RATIOS ---
  Stock P/E Debt/Equity ROCE % ROE %
0      None        None   None  None

--- SHAREHOLDING PATTERN ---
                                                                                               yearly
quarterly                                                                                            
{'': 'Promoters +', 'Mar 2023': 33.9, 'Jun 2023...  {'': 'Promoters +', 'Mar 2017': 31.35, 'Mar 20...
{'': 'FIIs +', 'Mar 2023': 20.62, 'Jun 2023': 2...  {'': 'FIIs +', 'Mar 2017': 14.11, 'Mar 2018': ...
{'': 'DIIs +', 'Mar 2023': 20.68, 'Jun 2023': 2...  {'': 'DIIs +', 'Mar 2017': 30.62, 'Mar 2018': ...
{'': 'Government +', 'Mar 2023': 0.16, 'Jun 202...  {'': 'Government +', 'Mar 2017': 0.01, 'Mar 20...
{'': 'Public +', 'Mar 2023': 24.63, 'Jun 2023':...  {'': 'Public +', 'Mar 2017': 23.91, 'Mar 2018'...
{'': 'No. of Shareholders', 'Mar 2023': 3644090...  {'': 'No. of Shareholders', 'Mar 2017': 844429...

--- PROS & CONS ---
Empty DataFrame
Columns: [Pros, 

In [49]:
# 1. FIX: SHAREHOLDING (Unpacking the nested 'quarterly' dictionaries)
sh_raw = stock_data.get('shareholding', [])
df_sh_initial = pd.DataFrame(sh_raw)

if not df_sh_initial.empty and 'quarterly' in df_sh_initial.columns:
    # Expand the 'quarterly' column into a new DataFrame
    df_shareholding = pd.DataFrame(list(df_sh_initial['quarterly']))

    # Set the first column (usually name like 'Promoters+') as index
    # We find the name column (often empty string '' or 'Name')
    name_col = [c for c in df_shareholding.columns if isinstance(c, str) and not any(char.isdigit() for char in c)][0]
    df_shareholding.set_index(name_col, inplace=True)
else:
    df_shareholding = pd.DataFrame()

df_shareholding.head()

,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Sep 2024,Dec 2024,Mar 2025,Jun 2025,Sep 2025,Dec 2025
,,,,,,,,,,,,
Promoters +,33.90,33.90,33.90,33.70,33.19,33.19,33.19,33.19,33.19,33.19,33.19,33.19
FIIs +,20.62,20.28,20.31,20.01,19.61,19.68,19.25,18.53,18.29,17.22,17.29,17.49
DIIs +,20.68,20.96,22.20,23.15,23.51,23.33,22.67,23.52,24.48,26.07,26.92,26.98
Government +,0.16,0.16,0.16,0.16,0.16,0.18,0.18,0.18,0.18,0.18,0.18,0.18
Public +,24.63,24.70,23.43,22.99,23.52,23.60,24.69,24.55,23.84,23.31,22.42,22.14


In [52]:
# 2. FIX: FUNDAMENTALS (Searching common Screener JSON paths)
# Sometimes these are in 'ratio_data' or 'key_metrics'
ratios_list = stock_data.get('ratios', [])
df_r = pd.DataFrame(ratios_list)
if not df_r.empty: df_r.set_index(df_r.columns[0], inplace=True)

def find_metric(name_list):
    """Checks top-level, then the Ratios table for a metric"""
    for name in name_list:
        if stock_data.get(name): return stock_data.get(name)
        if not df_r.empty and name in df_r.index: return df_r.loc[name].iloc[-1]
    return "N/A"

fundamentals = {
    "Stock P/E": find_metric(['stock_pe', 'Price to Earning', 'P/E']),
    "Debt/Equity": find_metric(['debt_to_equity', 'Debt to equity']),
    "ROCE %": find_metric(['roce', 'Return on capital employed']),
    "ROE %": find_metric(['roe', 'Return on equity']),
}

# 3. COMPUTE: Change in Promoter Holding (Last 2 available quarters)
change_promoter = "N/A"
if not df_shareholding.empty and any("Promoter" in idx for idx in df_shareholding.index):
    promoter_row = df_shareholding.loc[df_shareholding.index.str.contains("Promoter")].iloc[0]
    # Filter only for date columns
    dates = [c for c in promoter_row.index if "20" in str(c)]
    if len(dates) >= 2:
        change_promoter = promoter_row[dates[-1]] - promoter_row[dates[-2]]

print("\n--- FUNDAMENTAL RATIOS ---")
print(pd.DataFrame([fundamentals]))

print(f"\nChange in Promoter Holding (Latest Q): {change_promoter}%")


--- FUNDAMENTAL RATIOS ---
  Stock P/E Debt/Equity ROCE % ROE %
0       N/A         N/A    N/A   N/A

Change in Promoter Holding (Latest Q): 0.0%


In [53]:
# 1. Get the Quarters and Annual P&L DataFrames as they contain most ratios
df_q = pd.DataFrame(stock_data.get('quarters', [])).set_index('Metric') if stock_data.get('quarters') else pd.DataFrame()
df_ann = pd.DataFrame(stock_data.get('profit_and_loss', {}).get('annual_data', [])).set_index('Metric') if stock_data.get('profit_and_loss') else pd.DataFrame()

def extract_latest(df, metric_name):
    if df.empty or metric_name not in df.index:
        return "N/A"
    # Get the last column that isn't 'TTM' if possible, otherwise get the last one
    cols = [c for c in df.columns if c != 'TTM']
    val = df.loc[metric_name, cols[-1]]
    return val

# 2. Extracting from the data we HAVE
fundamentals = {
    "Stock P/E": extract_latest(df_q, 'P/E'), # Often found in Quarterly results
    "ROE %": extract_latest(df_ann, 'ROE'),   # Often at the bottom of P&L
    "ROCE %": extract_latest(df_ann, 'ROCE'),
    "Sales Growth": extract_latest(df_ann, 'Sales'),
}

# 3. If still N/A, check the 'warehouse' or 'ratios' key
if fundamentals["Stock P/E"] == "N/A":
    ratios_raw = stock_data.get('ratios', [])
    if ratios_raw:
        df_r = pd.DataFrame(ratios_raw).set_index('Metric')
        fundamentals["Debtor Days"] = extract_latest(df_r, 'Debtor Days')
        fundamentals["Inventory Days"] = extract_latest(df_r, 'Inventory Days')

print("--- RECOVERED RATIOS ---")
print(pd.DataFrame([fundamentals]))

--- RECOVERED RATIOS ---
  Stock P/E ROE % ROCE %  Sales Growth  Debtor Days  Inventory Days
0       N/A   N/A    N/A      218543.0            9             171


In [54]:
from bs4 import BeautifulSoup

In [55]:
def get_screener_header_ratios(ticker):
    url = f"https://www.screener.in{ticker}/consolidated/"
    headers = {'User-Agent': 'Mozilla/5.0'}

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        # Fallback to standalone page if consolidated isn't available
        url = f"https://www.screener.in{ticker}/"
        response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')

    ratios = {}
    # Screener stores top ratios in a <ul> with id 'top-ratios'
    top_ratios_list = soup.find('ul', {'id': 'top-ratios'})

    if top_ratios_list:
        for li in top_ratios_list.find_all('li'):
            name = li.find('span', {'class': 'name'}).text.strip()
            value = li.find('span', {'class': 'number'}).text.strip()
            # Clean up the name to match your requirements
            ratios[name] = value

    # Create DataFrame
    df = pd.DataFrame([ratios])

    # Filter for your specific requested columns
    target_cols = ['Stock P/E', 'Debt to equity', 'ROCE', 'ROE', 'Face Value', 'Dividend Yield']
    existing_cols = [c for c in target_cols if c in df.columns]

    return df[existing_cols]

In [56]:
# Execution
ticker = "TATASTEEL"
df_headers = get_screener_header_ratios(ticker)

print(f"--- Header Ratios for {ticker} ---")
print(df_headers)

ConnectionError: HTTPSConnectionPool(host='www.screener.intatasteel', port=443): Max retries exceeded with url: /consolidated/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d9964764a10>: Failed to resolve 'www.screener.intatasteel' ([Errno -2] Name or service not known)"))

okay leave it I have the balance sheet, cash flow statements, profit loss statements, and also share holding pattern, I want to now use an llm and give me a summary analysis of this where it tells me the following:-
1) pros and cons and

**Using the above data on LLM to get analysis**

In [ ]:
import openai

In [63]:
OPEN_AI_KEY = userdata.get('OPEN_AI_KEY')
if OPEN_AI_KEY:
    print("OPEN_AI_KEY value fetched")
else:
    print('Not able to fetch the key')

# Set your API Key
openai.api_key = OPEN_AI_KEY

OPEN_AI_KEY value fetched


In [74]:
def analyze_stock_with_llm(company_name, df_pl, df_bs, df_cf, df_sh, df_q):
    # Convert DataFrames to Markdown strings for the LLM to read easily
    data_payload = f"""
    Company Name: {company_name}

    1. PROFIT & LOSS STATEMENT (Annual):
    {df_pl.to_markdown()}

    2. BALANCE SHEET:
    {df_bs.to_markdown()}

    3. CASH FLOW STATEMENT:
    {df_cf.to_markdown()}

    4. SHAREHOLDING PATTERN:
    {df_sh.to_markdown()}

    5. QUARTERLY PERFORMANCE:
    {df_q.to_markdown()}
    """

    prompt = f"""
    You are an expert equity research analyst specializing in the Indian Stock Market.
    Analyze the provided financial data for {company_name} and provide a summary for a potential investor.

    Please provide the analysis under these 4 specific headings:

    1) STRENGTHS AND WEAKNESSES: Based strictly on the provided financial statements (P&L, Balance Sheet, Cash Flow, Shareholding).
    2) INVESTMENT OUTLOOK: Provide a perspective for:
       - Short-term (Few weeks)
       - Medium-term (Few months)
       - Long-term (Few years)
    3) INDUSTRY OUTLOOK: Provide the Global and Local (India-specific) outlook for the industry this stock belongs to.
    4) GOVERNMENT POLICY: Mention any specific Indian government policies (e.g., PLI schemes, Budget allocations, Export duties) that impact this sector.

    DATA:
    {data_payload}
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4.1-nano", # "gpt-4o",  # or "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You are a helpful and insightful financial analyst."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2 # Lower temperature for more factual analysis
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error connecting to LLM: {e}"

In [75]:
# --- EXECUTION ---
# Assuming you already have your DataFrames: df_pl, df_bs, df_cf, df_sh, df_quarters
analysis_report = analyze_stock_with_llm("Tata Steel Ltd", df_pl, df_bs, df_cf, df_shareholding, df_quarters)

print(analysis_report)

Certainly! Here's a comprehensive analysis of Tata Steel Ltd based on the provided financial data, structured under the four specified headings:

---

### 1) STRENGTHS AND WEAKNESSES

**Strengths:**
- **Revenue Stability & Scale:** Tata Steel has maintained substantial sales volumes, with recent annual sales around ₹2.2-2.3 lakh crore, indicating a large scale of operations.
- **Profitability Recovery:** After significant losses in earlier years (notably FY15-FY16), the company has shown a consistent return to profitability from FY19 onwards, with net profits reaching over ₹9,000 crore in FY24.
- **Operational Efficiency:** Operating profit margins have improved markedly, reaching around 13-14% in recent years, reflecting better cost management and operational efficiencies.
- **Strong Reserves & Equity Base:** Reserves have grown significantly, providing a buffer for future investments and downturns. Equity capital has remained stable, indicating no major dilution.
- **Cash Flow:** Pos

Observations and Changes to be done:
- Very long output, we want just few words in every point
- Govt Policy/Outlook for sector needs to be very short, put numbers in that if any
- No extra sentence like starting and ending



In [76]:
2+2

4

**Appendix**

**Tradingview Data Analysis**

In [28]:
from tradingview_screener import Query, Column

In [29]:
# 1. Define the financial fields you want to retrieve
# TradingView uses specific internal names for these metrics
financial_fields = [
    'name',
    'description',
    # Profit & Loss (Income Statement)
    'total_revenue', 'gross_profit', 'net_income', 'operating_margin',
    # Balance Sheet
    'total_assets', 'total_liabilities', 'total_debt', 'total_equity',
    # Cash Flow
    'cash_f_operating_activities', 'cash_f_investing_activities', 'cash_f_financing_activities', 'free_cash_flow'
]

# 2. Build the query for the Indian market (NSE or BSE)
# We filter for a specific ticker to get its unique data
ticker = "TATASTEEL"
query = (
    Query()
    .set_markets('india')
    .select(*financial_fields)
    .where(Column('name') == ticker)
)

# 3. Execute and process into a DataFrame
data, count = query.get_scanner_data()

if count > 0:
    # Convert list of rows to DataFrame
    df = pd.DataFrame(data)

    # Optional: Transpose for a "Statement" view (Metrics as rows)
    df_statement = df.transpose()
    df_statement.columns = [ticker]

    print(f"Financial Data for {ticker}:")
    print(df_statement)
else:
    print(f"No data found for ticker: {ticker}")

HTTPError: 400 Client Error: Bad Request
 Body: {"totalCount":0,"error":"Unknown field \"total_liabilities\"","data":null}
 for url: https://scanner.tradingview.com/india/scan

**Code for EODHD Data**

In [2]:
from google.colab import userdata

# Replace 'YOUR_SECRET_NAME' with the exact name you used in the Secrets panel
secret_key_value = userdata.get('EODHD')
if secret_key_value:
    print("Secret key value fetched")

Secret key value fetched


In [3]:
def fetch_eodhd_data(secret_key_value):
    api_key = secret_key_value
    if not api_key:
        print("No API key provided.")
        return

    # symbol = input("Enter symbol (e.g. RELIANCE.NS, TCS.NS, HDFCBANK.NS): ").strip().upper()
    # if "." not in symbol:
    #     symbol += ".NS"   # default to NSE (best for India)

    symbol = "RELIANCE.NS"

    # API base URL
    base = "https://eodhd.com/api"

    print(f"\nFetching data for {symbol}...\n")

    # 1. Fundamentals (all statements + ratios in one call)
    print("→ Fundamentals...")
    url = f"{base}/fundamentals/{symbol}?api_token={api_key}&fmt=json"
    r = requests.get(url, timeout=15)
    data = r.json()

    if "General" not in data:
        print("Error:", data.get("message", "Invalid key or symbol"))
        return

    gen = data.get("General", {})
    print(f"Company : {gen.get('Name')} ({gen.get('Code')})")
    print(f"Exchange: {gen.get('Exchange')} | Sector: {gen.get('Sector')}")

    fin = data.get("Financials", {})

    def latest_report(section, key):
        reports = fin.get(section, {}).get("yearly", [])
        if reports:
            latest = reports[0]
            return latest.get(key, "N/A")
        return "N/A"

    print("\nIncome Statement (Latest Annual)")
    print("  Revenue      :", latest_report("Income_Statement", "totalRevenue"))
    print("  Net Income   :", latest_report("Income_Statement", "netIncome"))
    print("  EPS          :", latest_report("Income_Statement", "eps"))

    print("\nBalance Sheet (Latest Annual)")
    print("  Total Assets :", latest_report("Balance_Sheet", "totalAssets"))
    print("  Total Debt   :", latest_report("Balance_Sheet", "totalDebt"))
    print("  Equity       :", latest_report("Balance_Sheet", "totalStockholderEquity"))

    print("\nCash Flow (Latest Annual)")
    print("  Operating CF :", latest_report("Cash_Flow", "operatingCashFlow"))
    print("  Investing CF :", latest_report("Cash_Flow", "investingCashFlow"))
    print("  Financing CF :", latest_report("Cash_Flow", "financingCashFlow"))

    # Key Ratios
    ratios = data.get("Highlights", {}) or data.get("Valuation", {})
    print("\nKey Ratios")
    print("  Trailing P/E :", ratios.get("trailingPE", "N/A"))
    print("  Forward P/E  :", ratios.get("forwardPE", "N/A"))
    print("  P/B          :", ratios.get("priceToBook", "N/A"))
    print("  ROE          :", ratios.get("returnOnEquity", "N/A"))
    print("  Debt/Equity  :", ratios.get("debtToEquity", "N/A"))

    # 2. Technical Indicators (latest values only)
    print("\n→ Technical Indicators (Latest)")
    indicators = [
        ("sma",  "SMA(50)",   {"function": "sma",  "period": 50,  "filter": "last_sma"}),
        ("ema",  "EMA(20)",   {"function": "ema",  "period": 20,  "filter": "last_ema"}),
        ("rsi",  "RSI(14)",   {"function": "rsi",  "period": 14,  "filter": "last_rsi"}),
        ("macd", "MACD",      {"function": "macd", "filter": "last_macd"})
    ]

    for code, name, params in indicators:
        url = f"{base}/technical/{symbol}"
        params["api_token"] = api_key
        params["fmt"] = "json"
        r = requests.get(url, params=params, timeout=10)
        res = r.json()

        if isinstance(res, dict) and "error" not in res:
            val = list(res.values())[0] if res else "N/A"
            print(f"  {name:10}: {val}")
        else:
            print(f"  {name:10}: Error fetching")

    print(f"\nDone for {symbol} at {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [7]:
if __name__ == "__main__":
    print("EODHD API - Fundamentals + Technicals Checker (India Stocks)\n")
    fetch_eodhd_data(secret_key_value)

EODHD API - Fundamentals + Technicals Checker (India Stocks)


Fetching data for RELIANCE.NS...

→ Fundamentals...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)